# Extract Info from postisrccd, calexp and calexpbg in Selected visits LSST in LSSTComCamSim

- author Sylvie Dagoret-Campagne
- creation date 2024-05-09
- last update 2024-05-09
- affiliation : IJCLab
- kernel : **w_2024_16**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS


import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'x-large'
plt.rcParams['axes.titlesize'] = 'x-large'
plt.rcParams['xtick.labelsize']= 'x-large'
plt.rcParams['ytick.labelsize']= 'x-large'

In [2]:
import gc

In [3]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [4]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [5]:
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.geom as geom

In [6]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [7]:
transform = AsinhStretch() + PercentileInterval(99.)

Import statements that we will need later

Let's make a new plot and metric tool, we'll base it on the example in the getting started guide.

In [8]:
# INSERT YOUR collection and tract
# for rehearsal use collection 2 which have CCDvisit
butlerRoot = "/repo/embargo"

collection1 = 'LSSTComCamSim/runs/nightlyvalidation/20240402/d_2024_03_29/DM-43612'
collection2 = 'LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612'
collection3 = 'LSSTComCamSim/runs/nightlyvalidation/20240404/d_2024_03_29/DM-43612'
collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240311/w_2024_10/PREOPS-4985'
collectionn = collection2
#collections = [collection1,collection2,collection3]
collections = [collection2]
collectionStr = collectionn.replace("/", "_")
fn_ccdVisit_tracts_patches = f"ccdVisittractpatch_{collectionStr}.csv"
instrument = 'LSSTComCamSim'
skymapName = "ops_rehersal_prep_2k_v1"
where_clause = "instrument = \'" + instrument+ "\'"

NDET = 9
suptitle=  collectionStr

tract = 9880 # mostly for light-curves
detector = 0
band = 'g'

suptitle = collectionStr + f" inst = {instrument} tract = {tract}"

In [9]:
#dataId = {"skymap": "latiss_v1", "tract": 5615, "instrument": "LATISS"}
dataId = {"skymap": skymapName, "tract": tract, "instrument": instrument}
repo = '/sdf/group/rubin/repo/oga/'
butler = dafButler.Butler(repo)
#t = Butler.get(table_sel, dataId=dataId, collections=collections)
registry = butler.registry

In [10]:
skymap = butler.get('skyMap', skymap=skymapName, collections=collections)

In [11]:
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # clear the figure
    plt.close(fig)  # close the figure
    gc.collect()    # call the garbage collector

In [12]:
def get_corners_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    corners_ra, corners_dec in decimal degrees
    """

    corners_x = [bbox.beginX, bbox.beginX, bbox.endX, bbox.endX]
    corners_y = [bbox.beginY, bbox.endY, bbox.endY, bbox.beginY]
    corners_ra = []
    corners_dec = []
    for i in range(4):
        radec = wcs.pixelToSky(corners_x[i], corners_y[i])
        corners_ra.append(radec.getRa().asDegrees())
        corners_dec.append(radec.getDec().asDegrees())
    
    return corners_ra, corners_dec

In [13]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

## Get pixel scale

In [14]:
import lsst.geom as geom
import lsst.sphgeom

skymap = butler.get('skyMap', skymap=skymapName, collections=collections )
tractInfo = skymap.generateTract(tract)
for patch in tractInfo:    
    patchID = patch.getSequentialIndex()
        
    ibb=patch.getInnerBBox()
    tWCS=tractInfo.getWcs()
       
    # loop on the 4 corners
    for icorn,corner in enumerate(ibb.getCorners()):
        p = geom.Point2D(corner.getX(), corner.getY())
        coord = tWCS.pixelToSky(p)


In [15]:
tWCS

FITS standard SkyWcs:
Sky Origin: (249.9173553719, +2.2314049587)
Pixel Origin: (14024, 14024)
Pixel Scale: 0.2 arcsec/pixel

In [16]:
#arcsec/pixel
pixel_scale = tWCS.getPixelScale().asArcseconds()

## Selected visits

In [17]:

# get the csv file produced by stat_on_visits_LSSTAuxtel.ipynb in ../Visits
#file_selected_visits = "../Visits/ccdVisittractpatch_LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240311_w_2024_10_PREOPS-4985.csv"
file_selected_visits = os.path.join("../Visits",fn_ccdVisit_tracts_patches)
df_myselectedvisits = pd.read_csv(file_selected_visits,index_col=0)
if 0:

    # select the band
    cut = (df_myselectedvisits.band == band) &  (df_myselectedvisits.tractID == tract) & (df_myselectedvisits.patchID == patch_sel)
    df_myselectedvisits = df_myselectedvisits[cut]

#move the visitid as a column not to loose it during the merge 
df_myselectedvisits.reset_index(inplace=True) 


In [18]:
index = 0
visitId = df_myselectedvisits.iloc[index]['visitId']
band = df_myselectedvisits.iloc[index]['band']
det = df_myselectedvisits.iloc[index]['detector']
dataId = {'visit': visitId, 'instrument':instrument , 'detector': det}

Note on the CCD visit here:
- zeroPoint is in ABMag relative to a flux of 1ADU/sec
- skyBg is in ADU , ADU/s , nJ per pixel ?
- skyNoise is in ADU, ADU/s , nJ  per pixel ?

In [19]:
df_myselectedvisits.iloc[index]

ccdVisitId     730865860864
visitId       7024040300001
band                      r
detector                  0
ra               124.617999
dec              -15.026212
llcra            124.458471
llcdec           -15.063877
ulcra            124.581823
ulcdec           -14.871498
urcra            124.777487
urcdec           -14.988434
lrcra            124.654231
lrcdec           -15.180921
ccdVid                    0
Vid                       0
nightObs           20240403
tractID                6914
patchID                 246
zeroPoint         31.258871
airmass            1.050426
skyBg             1703.7678
skyNoise           34.47179
expTime                30.0
Name: 0, dtype: object

## One postISRCCD and One Calexp

In [20]:
def ComputeStatfromImage(arr,remove_zero_flag=False):
    data_flat = arr.flatten()

    if remove_zero_flag:
        data_flat_nozero = data_flat[~(data_flat==0)]
        data_flat = data_flat_nozero 
        

    mu = np.mean(data_flat)
    med = np.median(data_flat)
    std = np.std(data_flat)
    sigMad = 1.4826 * np.median(np.fabs(data_flat - med))
    vmin,vmax = med - 3 * sigMad, med + 3 * sigMad
    return mu,med,std,sigMad,vmin,vmax


In [21]:
calexp = butler.get('calexp', **dataId,collections=collections)
bkgd  = butler.get('calexpBackground', **dataId,collections=collections)
# get the calibration constant  : expo_photocalibconstant_mean in nJ per ADU. To convert it in AB-Mag : convert_fluxtomag(expo_photocalibconstant_mean) --> zero point
calexp_info = calexp.getInfo()
photocalib = calexp_info.getPhotoCalib()
expo_photocalibconstant_mean = photocalib.getCalibrationMean()
expo_photocalibconstant_error = photocalib.getCalibrationErr()
calexp_md = calexp_info.getMetadata()
magzero,magzero_rms,magzero_nobj = calexp_md["MAGZERO"],calexp_md["MAGZERO_RMS"],calexp_md["MAGZERO_NOBJ"] 
bgmean,bgvar = calexp_md["BGMEAN"],calexp_md["BGVAR"]


mu_ce,med_ce,std_ce,sigMad_ce,vmin_ce,vmax_ce = ComputeStatfromImage(calexp.getImage().array,remove_zero_flag=False)
mu_bkg,med_bkg,std_bkg,sigMad_bkg,vmin_bkg,vmax_bkg = ComputeStatfromImage(bkgd.getImage().array,remove_zero_flag=False)

datasetRefs = registry.queryDatasets('postISRCCD', dataId=dataId, collections  = collections)
for i, ref in enumerate(datasetRefs):
    postisrccd  = butler.get(ref)
mu_pisr,med_pisr,std_pisr,sigMad_pisr,vmin_pisr,vmax_pisr = ComputeStatfromImage(postisrccd.getImage().array,remove_zero_flag=False)

mask = calexp.mask.array
the_mask = np.where(mask==0,1,0)

mu_ce_m,med_ce_m,std_ce_m,sigMad_ce_m,vmin_ce_m,vmax_ce_m = ComputeStatfromImage(calexp.getImage().array*the_mask ,remove_zero_flag=True)
mu_bkg_m,med_bkg_m,std_bkg_m,sigMad_bkg_m,vmin_bkg_m,vmax_bkg_m = ComputeStatfromImage(bkgd.getImage().array*the_mask,remove_zero_flag=True)
mu_pisr_m,med_pisr_m,std_pisr_m,sigMad_pisr_m,vmin_pisr_m,vmax_pisr_m = ComputeStatfromImage(postisrccd.getImage().array*the_mask,remove_zero_flag=True)

del calexp,bkgd,postisrccd,mask,the_mask

In [22]:
print(f"{index}) ======================{visitId} == {band}  == {det} =====================")
print('* photocalib ::')
print(photocalib)
print(expo_photocalibconstant_mean,convert_fluxtomag(expo_photocalibconstant_mean) )
print("magzero,magzero_rms,magzero_nobj :",magzero,magzero_rms,magzero_nobj)
print("bgmean,bgvar :",bgmean,bgvar)

print("* calexp ::")
print(mu_ce,med_ce,std_ce,sigMad_ce,vmin_ce,vmax_ce)
print(mu_ce_m,med_ce_m,std_ce_m,sigMad_ce_m,vmin_ce_m,vmax_ce_m)
print("* calexpBackground :: ")
print(mu_bkg,med_bkg,std_bkg,sigMad_bkg,vmin_bkg,vmax_bkg)
print(mu_bkg_m,med_bkg_m,std_bkg_m,sigMad_bkg_m,vmin_bkg_m,vmax_bkg_m)
print("* postisrccd :: ")
print(mu_pisr,med_pisr,std_pisr,sigMad_pisr,vmin_pisr,vmax_pisr)
print(mu_pisr_m,med_pisr_m,std_pisr_m,sigMad_pisr_m,vmin_pisr_m,vmax_pisr_m) 

0) ======================7024040300001 == r  == 0 =====================
* photocalib ::
spatially constant with mean: 1.12526 error: 0.000139179
1.12525625279675 31.27187141276424
magzero,magzero_rms,magzero_nobj : 27.5790682759651 0.000134291322191179 410
bgmean,bgvar : 1705.14601185246 0.822944011725373
* calexp ::
17.013908 1.0178335 560.0204 35.21349194641113 -104.62264236793519 106.65830931053162
0.015449499572078949 -0.08032277226448059 34.519808464181835 34.49981348835826 -103.57976323733925 103.41911769281029
* calexpBackground :: 
1704.2969 1704.4072 0.8708258 1.0149439453125 1701.3623947265626 1707.4520583984374
1704.2898291674942 1704.410888671875 0.8649762798814516 1.01566787109375 1701.3638850585937 1707.4578922851563
* postisrccd :: 
1721.3032 1705.3062 560.024 35.21971318359375 1599.6470127929688 1810.9652918945312
1704.3052786948663 1704.212890625 34.53072435273044 34.51189475097656 1600.6772063720703 1807.7485748779297


In [23]:
N = len(df_myselectedvisits)
NMAX = 0

In [24]:
df = pd.DataFrame(columns=['idx', 'visitId','ccdVisitId','band', 'detector',
                           'zeroPoint','airmass','skyBg','skyNoise','expTime',
                           'photocalib_m','photocalib_e',
                           'magzero','magzero_rms','magzero_nobj',
                           'bgmean','bgvar',
                           'mu_ce','med_ce','std_ce','sigMad_ce',
                           'mu_ce_m','med_ce_m','std_ce_m','sigMad_ce_m',
                           'mu_bkg','med_bkg','std_bkg','sigMad_bkg',
                           'mu_bkg_m','med_bkg_m','std_bkg_m','sigMad_bkg_m',
                           'mu_pisr','med_pisr','std_pisr','sigMad_pisr',
                           'mu_pisr_m','med_pisr_m','std_pisr_m','sigMad_pisr_m'
                          ])

In [ ]:
for index in range(N):

    visitId = df_myselectedvisits.iloc[index]['visitId']
    ccdvisitId = df_myselectedvisits.iloc[index]['ccdVisitId']
    band =  df_myselectedvisits.iloc[index]['band']
    det = df_myselectedvisits.iloc[index]['detector']     

    zeroPoint = df_myselectedvisits.iloc[index]['zeroPoint']
    airmass = df_myselectedvisits.iloc[index]['airmass']          
    skyBg  = df_myselectedvisits.iloc[index]['skyBg']
    skyNoise = df_myselectedvisits.iloc[index]['skyNoise']       
    expTime   = df_myselectedvisits.iloc[index]['expTime']

     

    dataId = {'visit': visitId, 'instrument':instrument , 'detector': det}
    
    
    calexp = butler.get('calexp', **dataId,collections=collections)
    bkgd  = butler.get('calexpBackground', **dataId,collections=collections)
    # get the calibration constant  : expo_photocalibconstant_mean in nJ per ADU. To convert it in AB-Mag : convert_fluxtomag(expo_photocalibconstant_mean) --> zero point
    calexp_info = calexp.getInfo()
    photocalib = calexp_info.getPhotoCalib()
    expo_photocalibconstant_mean = photocalib.getCalibrationMean()
    expo_photocalibconstant_error = photocalib.getCalibrationErr()
    calexp_md = calexp_info.getMetadata()
    magzero,magzero_rms,magzero_nobj = calexp_md["MAGZERO"],calexp_md["MAGZERO_RMS"],calexp_md["MAGZERO_NOBJ"] 
    bgmean,bgvar = calexp_md["BGMEAN"],calexp_md["BGVAR"]

    mu_ce,med_ce,std_ce,sigMad_ce,vmin_ce,vmax_ce = ComputeStatfromImage(calexp.getImage().array,remove_zero_flag=False)
    mu_bkg,med_bkg,std_bkg,sigMad_bkg,vmin_bkg,vmax_bkg = ComputeStatfromImage(bkgd.getImage().array,remove_zero_flag=False)

    datasetRefs = registry.queryDatasets('postISRCCD', dataId=dataId, collections  = collections)
    for i, ref in enumerate(datasetRefs):
        postisrccd  = butler.get(ref)
    mu_pisr,med_pisr,std_pisr,sigMad_pisr,vmin_pisr,vmax_pisr = ComputeStatfromImage(postisrccd.getImage().array,remove_zero_flag=False)

    mask = calexp.mask.array
    the_mask = np.where(mask==0,1,0)

    mu_ce_m,med_ce_m,std_ce_m,sigMad_ce_m,vmin_ce_m,vmax_ce_m = ComputeStatfromImage(calexp.getImage().array*the_mask ,remove_zero_flag=True)
    mu_bkg_m,med_bkg_m,std_bkg_m,sigMad_bkg_m,vmin_bkg_m,vmax_bkg_m = ComputeStatfromImage(bkgd.getImage().array*the_mask,remove_zero_flag=True)
    mu_pisr_m,med_pisr_m,std_pisr_m,sigMad_pisr_m,vmin_pisr_m,vmax_pisr_m = ComputeStatfromImage(postisrccd.getImage().array*the_mask,remove_zero_flag=True)

    del calexp,bkgd,postisrccd,mask,the_mask

    df.loc[index] = [index,visitId,ccdvisitId,band, det,
                     zeroPoint,airmass,skyBg,skyNoise,expTime,
                     expo_photocalibconstant_mean,expo_photocalibconstant_error,
                     magzero,magzero_rms,magzero_nobj,
                     bgmean,bgvar,
                     mu_ce,med_ce,std_ce,sigMad_ce,
                     mu_ce_m,med_ce_m,std_ce_m,sigMad_ce_m,
                     mu_bkg,med_bkg,std_bkg,sigMad_bkg,
                     mu_bkg_m,med_bkg_m,std_bkg_m,sigMad_bkg_m,
                     mu_pisr,med_pisr,std_pisr,sigMad_pisr,
                     mu_pisr_m,med_pisr_m,std_pisr_m,sigMad_pisr_m
                    ]
   


    if index<10 or index%100 == 0:
        print(f"{index}) ======================{visitId} == {band}  == {det} =====================")
        print('* photocalib ::')
        print(photocalib)
        print(expo_photocalibconstant_mean,convert_fluxtomag(expo_photocalibconstant_mean) )
        print("magzero,magzero_rms,magzero_nobj :",magzero,magzero_rms,magzero_nobj)
        print("bgmean,bgvar :",bgmean,bgvar)
        print("* calexp ::")
        print(mu_ce,med_ce,std_ce,sigMad_ce,vmin_ce,vmax_ce)
        print(mu_ce_m,med_ce_m,std_ce_m,sigMad_ce_m,vmin_ce_m,vmax_ce_m)
        print("* calexpBackground :: ")
        print(mu_bkg,med_bkg,std_bkg,sigMad_bkg,vmin_bkg,vmax_bkg)
        print(mu_bkg_m,med_bkg_m,std_bkg_m,sigMad_bkg_m,vmin_bkg_m,vmax_bkg_m)
        print("* postisrccd :: ")
        print(mu_pisr,med_pisr,std_pisr,sigMad_pisr,vmin_pisr,vmax_pisr)
        print(mu_pisr_m,med_pisr_m,std_pisr_m,sigMad_pisr_m,vmin_pisr_m,vmax_pisr_m) 

    if NMAX>0 and index>NMAX :
        break
   

0) ======================7024040300001 == r  == 0 =====================
* photocalib ::
spatially constant with mean: 1.12526 error: 0.000139179
1.12525625279675 31.27187141276424
magzero,magzero_rms,magzero_nobj : 27.5790682759651 0.000134291322191179 410
bgmean,bgvar : 1705.14601185246 0.822944011725373
* calexp ::
17.013908 1.0178335 560.0204 35.21349194641113 -104.62264236793519 106.65830931053162
0.015449499572078949 -0.08032277226448059 34.519808464181835 34.49981348835826 -103.57976323733925 103.41911769281029
* calexpBackground :: 
1704.2969 1704.4072 0.8708258 1.0149439453125 1701.3623947265626 1707.4520583984374
1704.2898291674942 1704.410888671875 0.8649762798814516 1.01566787109375 1701.3638850585937 1707.4578922851563
* postisrccd :: 
1721.3032 1705.3062 560.024 35.21971318359375 1599.6470127929688 1810.9652918945312
1704.3052786948663 1704.212890625 34.53072435273044 34.51189475097656 1600.6772063720703 1807.7485748779297
1) ======================7024040300001 == r  == 1 

In [ ]:
df

In [ ]:
if NMAX>0:
    outputfile = fn_ccdVisit_tracts_patches.split(".")[0]+f"_calexpinfo_{NMAX}.csv"
else:
    outputfile = fn_ccdVisit_tracts_patches.split(".")[0]+f"_calexpinfo.csv"
print(f"output file : {outputfile}")

In [ ]:
df.to_csv(outputfile)